## このスクリプト

教師が作った模範解答としての Excelファイルと、各学生が解答として提出したExcelファイルとを、セルを一つ一つ比較して、差がどれくらいあるかをチェックするスクリプトです。

## This script

This script compares the Excel file as the correct answer created by the teacher and the Excel file submitted as the answer by each student, cell by cell, and checks how much difference between those 2 files.

## Google Colab で実行

以下のコードを実行すると、Google Colab で実行している場合には Googleドライブがマウントされ使用できるようになります。
「このノートブックに Google ドライブのファイルへのアクセスを許可しますか?」というダイアログが表示されたら、 「Google ドライブに接続」をクリックして許可してください。

## When running in Google Colab

After running the following code, Google Drive will be mounted and available if you are running in Google Colab.
When the dialog "Do you want to allow this notebook to access files on Google Drive?" appears, click "Connect to Google Drive" to allow it.

## 提出Excelファイル毎について算出される指標
    
- **ind1** : 提出されたファイル内のセルのうち、正解ファイル内のそれに対応するセルとは値が異なるセルの数 （正解ファイルのセルが空白の場合は除く）

- **ind2** : 提出されたファイル内で、正解ファイルと同じ値を持つが、数字のみで構成されるセルの数
（つまり、学習者は、セルで Excel の計算機能を使用せずに、手動で値を計算した可能性がある）

- **ind3** : 正解ファイルでは空白だが、提出されたファイルでは空白ではないセルの数。

## Indicators to be calculated for each submitted Excel file
    
- **ind1** : how many cells in the submitted file are different in value from the corresponding cells in the correct answer file
   (excluding the case the cell in the correct answer file is blank)

- **ind2** : how many cells in the submitted file have the same value as the correct answer file but consist only of numbers
   (i.e., the learner possibly calculated the values manually, not using Excel's calculation feature at the cells)

- **ind3** : how many cells were blank in the correct answer file and non-blank in the submitted file.

In [ ]:
import sys
import os
import re
import glob
import datetime
import openpyxl
from openpyxl.comments import Comment
from openpyxl.styles import PatternFill, Border, Side, Alignment, Protection, Font

moduleList = sys.modules
ENV_COLAB = False
if 'google.colab' in moduleList:
    ENV_COLAB = True
    # print("google_colab")
if ENV_COLAB:
  from google.colab import drive
  drive.mount('/content/drive')
  gdpath = "/content/drive/MyDrive/"
else:
  gdpath = ""

# make folders
ansfile_folder = gdpath + "ediff-ans-files" # correct answer
stfile_folder  = gdpath + "ediff-st-files"  # students' files
out_folder     = gdpath + "ediff-out-files" # to save files 
os.makedirs(ansfile_folder, exist_ok=True)
os.makedirs(stfile_folder, exist_ok=True)
os.makedirs(out_folder, exist_ok=True)

ansfiles = glob.glob(ansfile_folder + "/**/*.xlsx", recursive=True)
for file0 in sorted(ansfiles):
  ansfilename = file0
print("answer file : " + ansfilename)
wb_ans      = openpyxl.load_workbook(ansfilename, data_only=True)

fileno = 0
xlist = []
ylist = []
textlist = []
colorlist = []
sizelist = []
participant_num_list = []
feedbacklist = []
stfiles = glob.glob(stfile_folder + "/**/*.xlsx", recursive=True)
for stfile in sorted(stfiles):
  fileno += 1
  # if(fileno > 8): ####
    # break ####
  matched = re.findall(r'[^0-9]([0-9]+)_assignsubmission_file', stfile)
  if matched:
    participant_num = matched[0]
  else:
    participant_num = ''
  print("now: " + stfile + " (" + participant_num + ")")
  wb_dataonly = openpyxl.load_workbook(stfile, data_only=True) # calc. value, not formula
  wb_dataonly.close()
  wb          = openpyxl.load_workbook(stfile)

  rownum_sum = 0
  for sheet in wb:
    rownum_sum += sheet.max_row
  print(f' rownum_sum:{rownum_sum}')
  if (rownum_sum > 10000000):
    nosave = True
  else:
    nosave = False
    
  ind1 = 0
  ind2 = 0
  ind3 = 0
  # for all the sheets
  for wsheet in wb:
    if (wsheet.title != 'enshuu' and wsheet.title != 'ouyou') :
      continue
    wsheet_ans      = wb_ans[wsheet.title]
    wsheet_dataonly = wb_dataonly[wsheet.title]
    #print(f'******* sheet name: {wsheet.title}') #####

    # all the cells in the worksheet (each row)
    irow = 0
    for row in wsheet.iter_rows():
      irow += 1
      if (irow > 100):
        continue
      icol = 0
      for cell in row:
        icol += 1
        if (icol > 100):
          continue
        value =                   wsheet.cell(cell.row, cell.column).value
        value_ans =           wsheet_ans.cell(cell.row, cell.column).value
        value_dataonly = wsheet_dataonly.cell(cell.row, cell.column).value
        if (str(value_dataonly) == str(value_ans)):
          if (str(value) == str(value_ans)): ## error! ->  re.match(r"^[+-]?[0-9]*[.]?[0-9]+$", value)
            ind2 += 1
        else:
        #if (str(cell.value) != str(value_ans)):
          cell.comment = Comment(str(value_ans), "excel-diff") # comment, author
          # print("[" + str(cell.row) + "," + str(cell.column) + "] " + str(value_dataonly) + "<" + str(cell.value) + ">  ans:" + str(value_ans))
          if(str(value_ans) == "None"):
            ind3 += 1
          else:
            ind1 += 1

  param_str = "ind1:" + str(ind1) + " ind2:" + str(ind2) + " ind3:" + str(ind3)
  print(stfile + "  " + param_str)
  outfile = stfile
  outfile = outfile.replace(stfile_folder, "").replace("/","_")
  if(nosave):
    print(" " + stfile + " is too big to save.")
  else:
    wb.save(out_folder + "/{:06}".format(ind1) + "_" + "{:06}".format(ind3) + "_" + outfile)
  wb.close()
  xlist.append(ind1)
  ylist.append(ind2)
  sizelist.append(15+ind3/20.0)
  textlist.append(str(fileno) + '<br>' + param_str + '<br>' + stfile)
  colorlist.append('rgb(' + str(110+ind1/4) + ', 54, 180)')
  participant_num_list.append(participant_num)
  feedbacklist.append(param_str)


In [ ]:
# Moodle grading worksheet line example: 
#"Participant 5219407",,"2023/01/16 03:19:34","ind1:0 ind2:123 ind3:20"
dtnow = datetime.datetime.now()
out_str = 'Identifier,Grade,"Last modified (grade)","Feedback comments"' + "\n"
for i in range(0,len(participant_num_list)):
    out_str += '"Participant ' + participant_num_list[i] \
               + '",,' + dtnow.strftime('%Y/%m/%d %H:%M:%S') + ',"' + feedbacklist[i] + '"' + "\n"
with open(out_folder + "/moodle_grading_worksheet.csv", mode='w') as f:
    f.write(out_str)

References:

https://it-ojisan.tokyo/colab-sys-modules/

https://tech-blog.rakus.co.jp/entry/20210729/openpyxl

https://atmarkit.itmedia.co.jp/ait/articles/2202/08/news031.html

https://plotly.com/python/


巨大なファイルを読み込むときのオプション

https://openpyxl.readthedocs.io/en/stable/optimized.html?highlight=load_workbook#read-only-mode


In [ ]:
# visualize
import plotly.graph_objects as go
fig = go.Figure(data=[go.Scatter(
    x = xlist, y = ylist, text = textlist,
    mode = 'markers',
    marker = dict( color = colorlist, size = sizelist, )
)])
fig.show()